In [8]:
import os
import json
import time
import requests
import random
import validators
import pickle

from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [3]:
# Read the search keywords and frontier URLs set
with open('../data/search_keywords.json', 'r') as fp:
    keywords = json.load(fp)

with open('../data/frontier_urls_initial.json', 'r') as fp:
    frontier_urls = json.load(fp)

print("Number of keywords: ", len(keywords))

Number of keywords:  4715


#### Creating frontier URLs list

We now expand the list of frontier URLs using the keywords

In [4]:
headers = {
    "Ocp-Apim-Subscription-Key": os.getenv("BING_API_KEY"),
    "X-Search-Location": "lat:48.31273;long:-9.03201;re:50"
}

def get_search_results(query):

    try:
        if "tübingen" not in query.lower():
            query += " Tübingen"

        params = {"q": query,  'responseFilter': 'Webpages', 'count': 20}
        response = requests.get(os.environ['BING_ENDPOINT'], headers=headers, params=params)
        search_results = response.json()

        # wait for 1 second to avoid rate limiting
        time.sleep(1)
        return search_results['webPages']['value']
    
    except Exception as e:
        print("Error: ", e)
        return []

search_results = get_search_results("library")
search_results

[{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
  'name': 'University Library | University of Tübingen - Universität Tübingen',
  'url': 'https://uni-tuebingen.de/en/facilities/university-library/',
  'isFamilyFriendly': True,
  'displayUrl': 'https://uni-tuebingen.de/en/facilities/university-library',
  'snippet': 'University of Tübingen. Geschwister-Scholl-Platz 72074 Tübingen Germany. Phone: +49 7071 29-0 Contact us',
  'dateLastCrawled': '2024-06-22T15:25:00.0000000Z',
  'cachedPageUrl': 'http://cc.bingj.com/cache.aspx?q=library+T%C3%BCbingen&d=4778765720097741&mkt=de-DE&setlang=de-DE&w=xc6z2rYVTSN9Ib0OTx2o0eZaGIkNWO1g',
  'language': 'en',
  'isNavigational': False,
  'noCache': False},
 {'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.1',
  'name': 'Universitätsbibliothek | Universität Tübingen',
  'url': 'https://uni-tuebingen.de/einrichtungen/universitaetsbibliothek/',
  'datePublished': '2024-06-04T00:00:00.0000000',
  'datePublishedDisplayText': '4. Juni 2

In [5]:
# Use multiprocessing to create 100 parallel processes to fetch search results
# map each keyword to get_search_results function

import multiprocess as mp

pool = mp.Pool(90)
search_results = pool.map(get_search_results, keywords)

Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'
Error:  'webPages'


In [6]:
print("Obtrained {} search results".format(len([x for x in search_results if x])))

Obtrained 4703 search results


In [10]:
keyword_results = zip(keywords, search_results)

# Save the search results to a pickle file
with open('../data/search_results.pkl', 'wb') as fp:
    pickle.dump(keyword_results, fp)

In [11]:
# combine the URLs from the search results and the frontier URLs

frontier_urls = set(frontier_urls)
for keyword, results in keyword_results:
    for result in results:
        url = result['url']
        frontier_urls.add(url)


print("Number of URLs in the frontier: ", len(frontier_urls))

Number of URLs in the frontier:  61935


In [12]:
# Write the updated frontier URLs to a pickle file

with open('../data/frontier_urls.pkl', 'wb') as fp:
    pickle.dump(frontier_urls, fp)

In [15]:
# count the number of URLs in result set
all_results = sum(search_results, [])

print("Total number of URLs in search results: ", len(all_results))
print("Number of unique URLs in search results: ", len(set([x['url'] for x in all_results])))

Total number of URLs in search results:  93972
Number of unique URLs in search results:  60917
